## Data Loader code

In [ ]:
import io
import pandas as pd
import requests
if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@data_loader
def load_data_from_api(*args, **kwargs):
    """
    Template for loading data from API
    """
    url = 'https://storage.googleapis.com/nyc_uber_data_analysis/uber_data.csv'
    response = requests.get(url)

    return pd.read_csv(io.StringIO(response.text), sep=',')


@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'


## Data Transformation code

In [ ]:
import pandas as pd
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@transformer
def transform(df, *args, **kwargs):
    """
    Template code for a transformer block.

    Add more parameters to this function if this block has multiple parent blocks.
    There should be one parameter for each output variable from each parent block.

    Args:
        data: The output from the upstream parent block
        args: The output from any additional upstream blocks (if applicable)

    Returns:
        Anything (e.g. data frame, dictionary, array, int, str, etc.)
    """
    # Specify your transformation logic here
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

    df = df.drop_duplicates().reset_index(drop=True)
    df['trip_id'] = df.index

    datetime_dim = df [['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop = True)

    datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
    datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
    datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
    datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
    datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
    datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

    datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
    datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
    datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
    datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
    datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
    datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
    datetime_dim['datetime_id'] = datetime_dim.index
    datetime_dim = datetime_dim[['datetime_id','tpep_pickup_datetime','pick_hour','pick_day','pick_month','pick_year',
                                'pick_weekday','tpep_dropoff_datetime','drop_hour','drop_day','drop_month','drop_year',
                                'drop_weekday']]
    
    passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
    passenger_count_dim['passenger_count_id'] = passenger_count_dim.index #primary key
    passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

    trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
    trip_distance_dim['trip_distance_id'] = trip_distance_dim.index #primary key
    trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]

    pickup_location_dim = df[['pickup_longitude','pickup_latitude']].reset_index(drop = True)
    pickup_location_dim['pickup_location_id'] = pickup_location_dim.index #primary key
    pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']]

    drop_location_dim = df[['dropoff_latitude','dropoff_longitude']].reset_index(drop=True)
    drop_location_dim['drop_location_id'] = drop_location_dim.index #primary key
    drop_location_dim = drop_location_dim[['drop_location_id','dropoff_latitude','dropoff_longitude']]

    payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
    }
    payment_type_dim = df[['payment_type']].reset_index(drop=True)
    payment_type_dim['payment_type_id'] = payment_type_dim.index #primary key
    payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
    payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

    rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
    }
    rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
    rate_code_dim['rate_code_id'] = rate_code_dim.index #primary key
    rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
    rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID', 'rate_code_name']]

    fact_table = df.merge(datetime_dim, left_on = 'trip_id', right_on = 'datetime_id')\
                .merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id')\
                .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id')\
                .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id')\
                .merge(drop_location_dim, left_on='trip_id', right_on='drop_location_id')\
                .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id')\
                .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id')\
                [['trip_id','VendorID','datetime_id','passenger_count_id','trip_distance_id', 'rate_code_id',
                  'store_and_fwd_flag','pickup_location_id','drop_location_id','payment_type_id','fare_amount',
                  'extra','tip_amount','tolls_amount','improvement_surcharge','total_amount']]
    
    return {"datetime_dim":datetime_dim.to_dict(orient="dict"),
       "passenger_count_dim":passenger_count_dim.to_dict(orient="dict"),
       "trip_distance_dim":trip_distance_dim.to_dict(orient="dict"),
       "pickup_location_dim":pickup_location_dim.to_dict(orient="dict"),
       "drop_location_dim":drop_location_dim.to_dict(orient="dict"),
       "payment_type_dim":payment_type_dim.to_dict(orient="dict"),
       "rate_code_dim":rate_code_dim.to_dict(orient="dict"),
       "fact_table":fact_table.to_dict(orient="dict")}


@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'


## Data Exporting to BigQuery

In [ ]:
from mage_ai.data_preparation.repo_manager import get_repo_path
from mage_ai.io.bigquery import BigQuery
from mage_ai.io.config import ConfigFileLoader
from pandas import DataFrame
from os import path

if 'data_exporter' not in globals():
    from mage_ai.data_preparation.decorators import data_exporter


@data_exporter
def export_data_to_big_query(data, **kwargs) -> None:
    """
    Template for exporting data to a BigQuery warehouse.
    Specify your configuration settings in 'io_config.yaml'.

    Docs: https://docs.mage.ai/design/data-loading#bigquery
    """
    config_path = path.join(get_repo_path(), 'io_config.yaml')
    config_profile = 'default'

    
    for key,value in data.items():
        table_id = 'uber-data-analytics-for-nyc.nyc_uber_data_analysis_suchi.{}'.format(key)
        BigQuery.with_config(ConfigFileLoader(config_path, config_profile)).export(
        DataFrame(value),
        table_id,
        if_exists='replace',  # Specify resolution policy if table name already exists
    )
